<h1><center>Notebook 4 : Dimensions computation </center></h1>

<h2>Instructions for this notebook</h2>


This notebook was written by Paul Platzer. It accompanies the following scientific publication:<br>
"Disentangling Density and Geometry in Weather Regime Dimensions using Stochastic Twins"<br>
By Paul Platzer, Bertrand Chapron, and Gabriele Messori.<br>
<b> add doi, date...etc. </b>

It is part of six notebooks that allow to reproduce the figures of the article. It must be used in conjunction with 500mb geopotential height data from ERA5 (last download from october 18th, 2023), as specified in the body text of the article.

What this notebook does:

<ol>
    <li> It opens raw ERA5 data and climatology, and recomputes anomaly as well as 10 days running-window time-smoothing.</li>
    <li> It opens preprocessed ERA5 data (10day-smoothed anomalies projected on winter-time EOFs), the results of the GMM fit, and the stochastic twins.</li>
    <li> It computes weather-regime indices based on ERA5 projected data, stochastic twin data, and GMM fit parameters. </li>
    <li> It computes dimension in various ways, with either only ERA5 data (projected or not on EOFs), or with ERA5 and stochastic twins, or with only stochastic twins, as outlined in the paper. </li>
    <li> It saves computed dimensions. </li>
</ol>

To use properly this notebook, you must:

<ol>
    <li> Have previously run "Notebook0-ERA5_pretreatment.ipynb", "Notebook1-GMM_fit.ipynb" and "Notebook2-Stochastic_twin_generation.ipynb".</li>
    <li> Run the whole notebook once.</li>
    <li> If you have already run the notebook, and you just want to replot the figures, you can skip section III. </li>
</ol>

Note that, for this code to work on your machine, you should:

<ol>
    <li> Have download the libraries listed in the first two cells. </li>
    <li> Use python 3 : this code was tested using python 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0] </li>
    <li> Have enough memory and computing resources. If not, you might have to modify the code to make it work. The code was run using a Dell Inc. Precision 7550 which has 33G of RAM and setting the same amount of swap space, and for processing we have Intel® Core™ i7-10875H CPU @ 2.30GHz × 16, with graphics card NVIDIA Corporation TU104GLM [Quadro RTX 4000 Mobile / Max-Q] / Mesa Intel® UHD Graphics (CML GT2). </li>
</ol>

In [1]:
import sys
print(sys.version)

3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]


In [1]:
import numpy as np
import xarray as xr
import pandas as pd
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm
from sklearn.neighbors import KDTree, NearestNeighbors
from sklearn.metrics import pairwise_distances
import sys
sys.path.append('../functions/.')
from functions import locdim

In [2]:
cols = ['k', '#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']
import matplotlib
matplotlib.rcParams.update({'font.size': 14})

In [3]:
data_folder = '../outputs/data/'
figure_folder = '../outputs/figures/'

# I. Load ERA5 + climatology and recompute anomaly

In [4]:
ERA5_folder = '/home/pplatzer/Documents/pro/postdoc/DATA/ERA5/'
file = 'ERA5_z500_1979_2023.grib'

In [5]:
ds = xr.open_dataset(ERA5_folder+file, engine='cfgrib')

Load climatology.

In [6]:
ds_clim = xr.open_dataset(ERA5_folder + 'ERA5_z500_clim_1979_2023.nc')

Compute anomaly (may take some time).

In [7]:
ds = ds.assign_coords(dayofyear=ds['time'].dt.dayofyear)

In [8]:
z_anom = ds['z'].copy()
dt_comp = np.timedelta64(int(2*365),'D')
t0 = ds.time[0]
while t0 < ds.time[-1]:
    t1 = t0.copy() + dt_comp
    print('Computing anomaly from time '+str(np.array(t0))+' to '+str(np.array(t1)))
          
    z_anom.loc[t0:t1] = ( ds['z'].sel(time=slice(t0,t1)) - ds_clim['z'].sel(dayofyear=ds['dayofyear'].sel(time=slice(t0,t1))) ).values
    t0 += dt_comp

Computing anomaly from time 1979-01-01T00:00:00.000000000 to 1980-12-31T00:00:00.000000000
Computing anomaly from time 1980-12-31T00:00:00.000000000 to 1982-12-31T00:00:00.000000000
Computing anomaly from time 1982-12-31T00:00:00.000000000 to 1984-12-30T00:00:00.000000000
Computing anomaly from time 1984-12-30T00:00:00.000000000 to 1986-12-30T00:00:00.000000000
Computing anomaly from time 1986-12-30T00:00:00.000000000 to 1988-12-29T00:00:00.000000000
Computing anomaly from time 1988-12-29T00:00:00.000000000 to 1990-12-29T00:00:00.000000000
Computing anomaly from time 1990-12-29T00:00:00.000000000 to 1992-12-28T00:00:00.000000000
Computing anomaly from time 1992-12-28T00:00:00.000000000 to 1994-12-28T00:00:00.000000000
Computing anomaly from time 1994-12-28T00:00:00.000000000 to 1996-12-27T00:00:00.000000000
Computing anomaly from time 1996-12-27T00:00:00.000000000 to 1998-12-27T00:00:00.000000000
Computing anomaly from time 1998-12-27T00:00:00.000000000 to 2000-12-26T00:00:00.000000000

Time-smoothing (may take some time).

In [24]:
za_smooth = z_anom.rolling(time=2*10, center=True).mean().dropna('time') # 10 days rolling average

# II. Load preprocessed ERA5, GMM parameters, and stochastic twins

In [9]:
# load
smooth = True
npzfile = np.load(data_folder + 'GMM_params_' + smooth*'smooth' + (1-smooth)*'unsmooth' + '.npz')
means = npzfile['means']
covs = npzfile['covs']
weights = npzfile['weights']
nclus = npzfile['nclus']
cov_type = npzfile['cov_type']
ndim_gmm = npzfile['ndim_gmm']
random_state_gmm = npzfile['random_state_gmm']
regime_names = npzfile['regime_names']
regime_short_names = npzfile['regime_short_names']
regime_attribution = npzfile['regime_attribution']
regime_attribution_distance = npzfile['regime_attribution_distance']
Ndays = npzfile['Ndays']
Ndays_kmeans = npzfile['Ndays_kmeans']
Ndays_tot = npzfile['Ndays_tot']

In [10]:
allpcs = xr.open_dataset(data_folder + 'pcs_' + smooth*'smooth' + (1-smooth)*'unsmooth' + '.nc')['pseudo_pcs']
eofs = xr.open_dataset(data_folder + 'eofs_' + smooth*'smooth' + (1-smooth)*'unsmooth' + '.nc')['eofs']
pourc_EOF = xr.open_dataset(data_folder + 'pourc_eofs_' + smooth*'smooth' + (1-smooth)*'unsmooth' + '.nc')['variance_fractions']

In [11]:
# normalize pcs but keep relative variances
pcs_norm = allpcs / (allpcs.sel(mode=0)).std(dim='time')

In [12]:
npzfile = np.load(data_folder + 'stoch_twins.npz')
X = npzfile['X']
ndim_gmm = npzfile['ndim_gmm']
rho = npzfile['rho']
scaling = npzfile['scaling']
Ntwins = X.shape[0]
L = X.shape[1]

# III. Compute weather regime index (WRI)

In [13]:
index = np.zeros( ( Ntwins , L , len(means) ) ,  dtype='float64')
for k in range(Ntwins):
    for i in range(len(means)):
        index[k,:,i] = np.sum( X[k][:,:ndim_gmm]*means[i] , axis=1)
        index[k,:,i] -= np.mean( index[k,:,i] )
        index[k,:,i] /= np.std( index[k,:,i] )    

In [14]:
c_index = np.zeros( ( Ntwins , L , len(means) ) ,  dtype='float64')
for k in range(Ntwins):
    for i in range(len(means)):
        c_index[k,:,i] = np.einsum( 'ij,jk,ik->i' , X[k][:,:ndim_gmm]-means[i] ,
                                   np.linalg.inv(covs[i]) , X[k][:,:ndim_gmm]-means[i] )

In [15]:
index_nat = np.zeros( ( len(pcs_norm), len(means) ) ,  dtype='float64')
for i in range(len(means)):
    index_nat[:,i] = np.sum( np.array(pcs_norm[:,:ndim_gmm])*means[i] , axis=1)
    index_nat[:,i] -= np.mean( index_nat[:,i] )
    index_nat[:,i] /= np.std( index_nat[:,i] )

In [16]:
c_index_nat = np.zeros( ( len(pcs_norm) , len(means) ) ,  dtype='float64')
for i in range(len(means)):
    c_index_nat[:,i] = np.einsum( 'ij,jk,ik->i' , pcs_norm[:,:ndim_gmm]-means[i] ,
                                 np.linalg.inv(covs[i]) , pcs_norm[:,:ndim_gmm]-means[i] )

# IV. Compute dimensions

### IV.A. Using only natural ERA5 data
First whole z500 fields

In [25]:
# za_djf = np.reshape( np.array(z_anom.isel(time = (z_anom.time.dt.season == 'DJF'))),
#                    (np.array(z_anom.time.dt.season == 'DJF').sum(),len(z_anom.latitude)*len(z_anom.longitude)))
za_djf = np.reshape( np.array(za_smooth.isel(time = (za_smooth.time.dt.season == 'DJF'))),
                   (np.array(za_smooth.time.dt.season == 'DJF').sum(),len(za_smooth.latitude)*len(za_smooth.longitude)))

In [26]:
# find analogues with KDTree
Klarge = 500
nn = NearestNeighbors(n_neighbors = Klarge).fit( za_djf )
dist_nat, ind_nat = nn.kneighbors( za_djf , return_distance=True)

In [27]:
# define multiple Ks for the computation of dimensions
K = np.arange(150 , Klarge + 10 , 10)

dd_nat = np.full( ( L , len(K) ) , np.nan )
for i_K in tqdm(range(len(K))):
    dd_nat[:,i_K] = locdim(dist_nat[:,1:1+K[i_K]])

  0%|          | 0/36 [00:00<?, ?it/s]

Then projected data.

In [28]:
# find neighbours using KDTree
nn = NearestNeighbors(n_neighbors = Klarge).fit( pcs_norm )
dist_pcs, ind_pcs = nn.kneighbors( pcs_norm , return_distance=True)

In [29]:
dd_pcs = np.full( ( L , len(K) ) , np.nan )
for i_K in tqdm(range(len(K))):
    dd_pcs[:,i_K] = locdim(dist_pcs[:,1:1+K[i_K]])

  0%|          | 0/36 [00:00<?, ?it/s]

### Save

In [30]:
# save if needed
np.savez(data_folder + 'dim_nat.npz',
        dd_nat = dd_nat,
        dd_pcs = dd_pcs,
         K = K,
        )

In [16]:
# load if needed
# npzfile = np.load(data_folder + 'dim_nat.npz')
# dd_nat = npzfile['dd_nat']
# dd_pcs = npzfile['dd_pcs']
# K = npzfile['K']

## IV.A. Stochastic twin analogues
First, as target *and* analogues.<br>
Then, only as analogues.

In [31]:
# first find neighbours
DIST_twin = []; IND_twin = []
DIST_nat_twin = []; IND_nat_twin = []

for k in tqdm(range(Ntwins)):
    nn = NearestNeighbors(n_neighbors = Klarge+1 ).fit( X[k] )
    # twins as target and analogues or as analogues
    for i_type in range(2):
        dist, ind = nn.kneighbors( [ X[k] , pcs_norm ][i_type] , return_distance=True)
        if i_type == 0:
            DIST_twin.append( dist )
            IND_twin.append( ind )
        else:
            DIST_nat_twin.append( dist )
            IND_nat_twin.append( ind )

  0%|          | 0/20 [00:00<?, ?it/s]

In [32]:
# then compute dimensions
dd_twin = np.full( ( Ntwins , L , len(K) ) , np.nan )
dd_nat_twin = np.full( ( Ntwins , L , len(K) ) , np.nan )

for k in tqdm(range(Ntwins)):
    for i_K in tqdm(range(len(K))):
        dd_twin[k,:,i_K] = locdim(DIST_twin[k][:,1:1+K[i_K]])        
        dd_nat_twin[k,:,i_K] = locdim(DIST_nat_twin[k][:,0:K[i_K]])        

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

...

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

### Save

In [33]:
# save if needed
np.savez(data_folder + 'dim_twins.npz',
        dd_nat_twin = dd_nat_twin,
        dd_twin = dd_twin,
        K = K,
        )

In [17]:
# load if needed
# npzfile = np.load(data_folder + 'dim_twins.npz')
# dd_nat_twin = npzfile['dd_nat_twin']
# dd_twin = npzfile['dd_twin']
# K = npzfile['K']